In [12]:
import dgl
import dgl.data
import dgl.nn as gnn

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

Датасет Cora: 
* узлы - статьи
* связи - цитирование одной статьей другой
* каждый узел в качестве фичей содержит нормализованный word count vector 

Датасет может состоять из одного или нескольких графов. Cora состоит из одного.

Граф в DGL может хранить фичи для узлов и ребер в виде словарей `ndata` и `edata`. 

Фичи узлов в Cora:
* x_mask - булев тензор, показывающий, входит ли узел в множество x (train, val, test)
* label - метка узла
* feat - фичи узла

In [8]:
dataset = dgl.data.CoraGraphDataset()
G = dataset[0]

print(f"Кол-во категорий: {dataset.num_classes}")

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Кол-во категорий: 7


In [10]:
class GCN(nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super().__init__()
        self.conv1 = gnn.GraphConv(n_input, n_hidden)
        self.conv2 = gnn.GraphConv(n_hidden, n_output)
    
    def forward(self, G, in_features):
        out = F.relu(self.conv1(G, in_features))
        out = self.conv2(G, out)
        return out

In [13]:
n_input = G.ndata['feat'].shape[1]
n_hidden = 16
n_out = dataset.num_classes
n_epochs = 100

model = GCN(n_input, n_hidden, n_out)

In [15]:
# check
model(G, G.ndata['feat']).shape

torch.Size([2708, 7])

In [18]:
model = GCN(n_input, n_hidden, n_out)

optimizer = optim.Adam(model.parameters(), lr=.01)
criterion = nn.CrossEntropyLoss()

best_val_acc, best_test_acc = 0, 0


features = G.ndata['feat']
labels = G.ndata['label']
train_mask = G.ndata['train_mask']
val_mask = G.ndata['val_mask']
test_mask = G.ndata['test_mask']

for epoch in range(n_epochs):
    # forward
    logits = model(G, features)
    
    # loss
    loss = criterion(logits[train_mask], labels[train_mask])

    # backward
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # eval
    with torch.no_grad():
        predictions = logits.argmax(dim=1)
        train_acc = (predictions[train_mask] == labels[train_mask]).float().mean()
        val_acc = (predictions[val_mask] == labels[val_mask]).float().mean()
        test_acc = (predictions[test_mask] == labels[test_mask]).float().mean()

        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

    if not epoch % 5:
        print(f'In epoch {epoch}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})')

In epoch 0, loss: 1.945, val acc: 0.172 (best 0.172), test acc: 0.159 (best 0.159)
In epoch 5, loss: 1.886, val acc: 0.516 (best 0.540), test acc: 0.527 (best 0.546)
In epoch 10, loss: 1.804, val acc: 0.590 (best 0.590), test acc: 0.612 (best 0.612)
In epoch 15, loss: 1.699, val acc: 0.622 (best 0.622), test acc: 0.631 (best 0.631)
In epoch 20, loss: 1.572, val acc: 0.636 (best 0.636), test acc: 0.646 (best 0.646)
In epoch 25, loss: 1.425, val acc: 0.644 (best 0.646), test acc: 0.657 (best 0.656)
In epoch 30, loss: 1.263, val acc: 0.658 (best 0.658), test acc: 0.673 (best 0.668)
In epoch 35, loss: 1.094, val acc: 0.692 (best 0.692), test acc: 0.694 (best 0.694)
In epoch 40, loss: 0.927, val acc: 0.704 (best 0.704), test acc: 0.712 (best 0.712)
In epoch 45, loss: 0.771, val acc: 0.718 (best 0.718), test acc: 0.725 (best 0.723)
In epoch 50, loss: 0.632, val acc: 0.730 (best 0.730), test acc: 0.740 (best 0.740)
In epoch 55, loss: 0.515, val acc: 0.754 (best 0.754), test acc: 0.761 (best 0

In [19]:
torch.cuda.is_available()

False

In [20]:
labels.shape

torch.Size([2708])